经验证：
* 1、test的时间顺序并不是全部都是时序的，排行榜是按时序排列后取前0.78为public，后0.22为private，所以以后的工作重心将落到train的后44%
* 2、测试集中私有部分从2018-07-24 09:00:00开始，有9173472条数据，包含全部的building_id
* 3、测试集中公共部分与私有部分对应的数据从2017-07-24 09:00:00开始，有9174900条数据，包含全部的building_id
* 4、重点为训练集中2017-07-24 09:00:00后的数据建模，有9003109条数据，占训练集0.445
* 5、为防止测试集中不评分的那部分不知道算不算在0.22里面，也为防止一定的过拟合，可以考虑重点关注2017-07月后的训练数据，占0.51
* 6、验证EDA1后，发现0-104号建筑5月20日前的大量为0的数据是有误的，训练集不应该对这部分数据进行过拟合，删除这部分数据训练对提高public评分很有帮助，遗憾的是，private并不包含这类数据
* 7、猜想某些大范围为0的数据都同0-104号一样是有误的，重点验证7月后的该种数据，测试是否能提高public

In [5]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')
import gc


# matplotlib and seaborn for plotting
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
import matplotlib.patches as patches

from plotly import tools, subplots
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.express as px
pd.set_option('max_columns', 100)


py.init_notebook_mode(connected=True)
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
import plotly.graph_objs as go

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('../input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

../input/ashrae-energy-prediction/train.csv
../input/ashrae-energy-prediction/sample_submission.csv
../input/ashrae-energy-prediction/building_metadata.csv
../input/ashrae-energy-prediction/weather_train.csv
../input/ashrae-energy-prediction/weather_test.csv
../input/ashrae-energy-prediction/ashrae-energy-prediction.zip
../input/ashrae-energy-prediction/test.csv


In [6]:
# 导入数据, 只有train 和 test
train = pd.read_csv('../input/ashrae-energy-prediction/train.csv')
test = pd.read_csv('../input/ashrae-energy-prediction/test.csv')

In [7]:
# 时间戳转换日期
train["timestamp"] = pd.to_datetime(train["timestamp"])
test["timestamp"] = pd.to_datetime(test["timestamp"])

In [8]:
train_exception = pd.read_pickle('../output/fork-of-ashrae-eda-exception-label2/train_exception.pkl')
train['exception1'] = train_exception.exception.values
del train_exception
gc.collect()

train_exception = pd.read_pickle('../output/ashrae-eda-exception-label5/train_exception.pkl')
train['exception'] = train_exception.exception.values
del train_exception
gc.collect()

0

In [9]:
test_exception = pd.read_pickle('../output/fork-of-ashrae-eda-exception-label2/test_exception.pkl')
test['exception1'] = test_exception.exception.values
del test_exception
gc.collect()

test_exception = pd.read_pickle('../output/ashrae-eda-exception-label5/test_exception.pkl')
test['exception'] = test_exception.exception.values
del test_exception
gc.collect()

0

In [10]:
gc.collect()

20

In [11]:
train.exception.value_counts(dropna=False)

 0    16258788
-1     1923669
-3     1051924
 3      452505
 1      365029
-2       85465
 2       78720
Name: exception, dtype: int64

In [12]:
train.exception1.value_counts(dropna=False)

 0    17761403
-1     1924531
 1      365029
-2       85465
 2       79672
Name: exception1, dtype: int64

In [13]:
test.exception.value_counts(dropna=False)

 0.0    32438494
-1.0     3904132
-3.0     2171394
 NaN     1205302
 3.0      912126
 1.0      771056
-2.0      147834
 2.0      147262
Name: exception, dtype: int64

In [14]:
test.exception1.value_counts(dropna=False)

 0.0    35435620
-1.0     3920902
 NaN     1239910
 1.0      771806
-2.0      170502
 2.0      158860
Name: exception1, dtype: int64

In [15]:
test[(test.building_id == 235) & (test.meter == 1)].exception.value_counts(dropna=False)

-3.0    15794
 3.0     1726
Name: exception, dtype: int64

In [16]:
test[(test.building_id == 235) & (test.meter == 1)].exception1.value_counts(dropna=False)

-1.0    16770
 1.0      750
Name: exception1, dtype: int64

In [17]:
updata_id = test[(test.building_id == 235) & (test.meter == 1) & (test.exception1 == 1)].row_id.values

In [18]:
len(updata_id)

750

In [19]:
test.loc[updata_id, 'exception'] = 1

In [20]:
test[(test.building_id == 235) & (test.meter == 1)].exception.value_counts(dropna=False)

-3.0    15044
 3.0     1726
 1.0      750
Name: exception, dtype: int64

In [21]:
test.exception.value_counts(dropna=False)

 0.0    32438494
-1.0     3904132
-3.0     2170644
 NaN     1205302
 3.0      912126
 1.0      771806
-2.0      147834
 2.0      147262
Name: exception, dtype: int64

In [22]:
# ## 存疑异常标记

# # 30前期有大量0值
# train.loc[(train.building_id == 30) & (train.meter == 1) & (train.meter_reading < 212), 'exception'] = 2

# # 43大量0值
# train.loc[(train.building_id == 43) & (train.meter == 1) & (train.meter_reading == 0), 'exception'] = 2

# # 28大量0值
# train.loc[(train.building_id == 28) & (train.meter == 1) & (train.meter_reading == 0), 'exception'] = 2

In [23]:
gc.collect()

160

In [24]:
train[['exception']].to_pickle('../output/fork-of-ashrae-eda-exception-label5/train_exception.pkl')
test[['exception']].to_pickle('../output/fork-of-ashrae-eda-exception-label5/test_exception.pkl')